<em><sub>This page is available as an executable or viewable <strong>Jupyter Notebook</strong></sub></em>
<br/><br/>
<a href="https://mybinder.org/v2/gh/avan1235/KotlinDL/notebooks?filepath=docs%2Ftransfer_learning.ipynb"
   target="_parent">
   <img align="left"
        src="https://mybinder.org/badge_logo.svg"
        height="20">
</a>
<a href="https://nbviewer.jupyter.org/github/avan1235/KotlinDL/blob/notebooks/docs/transfer_learning.ipynb"
   target="_parent">
   <img align="right"
        src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg"
        height="20">
</a>
<br/><br/>

In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlin-deeplearning-api:0.2.0")

# Transfer learning with KotlinDL

Transfer learning is a popular deep learning technique that allows you to save time on training a neural network while achieving great performance too. 
It leverages existing pre-trained models and allows you to tweak them to your task by training only a part of the neural network.

In this tutorial, we will take a pre-trained VGG-19 Keras model that has been trained on a large dataset (ImageNet) to classify color images into 1000 categories. 
We will load this model with KotlinDL, and fine-tune it by training only some of its layers to classify images from our own dataset.

## Data preparation
For the purposes of this tutorial, we downloaded a dataset containing images of dogs and cats 
(it's a subset of the dataset from the famous Kaggle competition with only 50 images per class instead of 12500 images per class like the original dataset). 
Next, we resized the images to be 224 x 224 pixels. This is the same image size as VGG-19 was trained on. 
It is important for all the images to be the same size, however, 
it is not critical for them to be of the exact size as what the model was trained on – if needed, we can replace the input layer.

We've stored the image dataset so each class is in its own folder: 
```
small-catdogs/
    cat/
    dogs/
```
There are, of course, many ways to organize your own dataset. 
This way makes it easier to get the labels for all the examples based on the folder names 
(we assume that the folder with the name _cat_ contains images of cats).

Now we need to create a `Dataset` from these images. 
You can do so via the Image Preprocessing Pipeline description, and building a dataset from those. 

Here's code that will go through a folder structure received via ```catDogsSmallDatasetPath()```, loads and resizes the images, and applies the VGG-19 specific preprocessing.

In [2]:
import org.jetbrains.kotlinx.dl.api.inference.keras.loaders.ModelType
import org.jetbrains.kotlinx.dl.dataset.OnFlyImageDataset
import org.jetbrains.kotlinx.dl.dataset.dogsCatsSmallDatasetPath
import org.jetbrains.kotlinx.dl.dataset.image.ColorOrder
import org.jetbrains.kotlinx.dl.dataset.preprocessor.*
import org.jetbrains.kotlinx.dl.dataset.preprocessor.generator.FromFolders
import org.jetbrains.kotlinx.dl.dataset.preprocessor.image.InterpolationType
import org.jetbrains.kotlinx.dl.dataset.preprocessor.image.load
import org.jetbrains.kotlinx.dl.dataset.preprocessor.image.resize
import java.io.File

val NUM_CHANNELS = 3L
val IMAGE_SIZE = 224L
val TRAIN_TEST_SPLIT_RATIO = 0.7
val TRAINING_BATCH_SIZE = 8
val TEST_BATCH_SIZE = 16
val EPOCHS = 3

val dogsVsCatsDatasetPath = dogsCatsSmallDatasetPath()

val preprocessing = preprocess {
    transformImage {
        load {
            pathToData = File(dogsVsCatsDatasetPath)
            imageShape = ImageShape(channels = NUM_CHANNELS)
            colorMode = ColorOrder.BGR
            labelGenerator = FromFolders(mapping = mapOf("cat" to 0, "dog" to 1))
        }
        resize {
            outputHeight = IMAGE_SIZE.toInt()
            outputWidth = IMAGE_SIZE.toInt()
            interpolation = InterpolationType.BILINEAR
        }
    }
    transformTensor {
        sharpen {
            modelType = ModelType.VGG_19
        }
    }
}

val dataset = OnFlyImageDataset.create(preprocessing).shuffle()
val (train, test) = dataset.split(TRAIN_TEST_SPLIT_RATIO)

In the final lines, after creating a dataset, we shuffle the data, so that when we split it into training and testing portions, we do not get a test set containing only images of one class.    
 
## VGG-19
KotlinDL bundles a lot of pre-trained models available via ModelZoo object. 
You can either train a model from scratch yourself and store it for later use on other tasks, or you can import a pre-trained Keras model with compatible architecture.  

In this tutorial, we will load VGG-19 model and weights that are made available in the Model Zoo: 

In [3]:
import org.jetbrains.kotlinx.dl.api.core.Sequential
import org.jetbrains.kotlinx.dl.api.inference.keras.loaders.ModelZoo


val modelZoo = ModelZoo(commonModelDirectory = File("cache/pretrainedModels"),
                        modelType = ModelType.VGG_19)
val model = modelZoo.loadModel() as Sequential

## Transfer Learning
Now we have created the dataset, and we have a model, we can put everything together, and apply the transfer learning technique.

At this point, we need to decide which layers of this model we want to fine-tune, which ones we want to leave as is and if we want to add or remove layers. 
You can use `model.summary()` to inspect the model's architecture.

This model consists mainly of Conv2D and MaxPool2D layers and has a couple of dense layers at the end. One way to do transfer learning (although, of course, not the only one) is to leave the convolutional layers as they are, and re-train the dense layers. 
So this is what we will do:
- We'll "freeze" all the Conv2D and MaxPool2D layers – the weights for them will be loaded, but they will not be trained any further.
- The last layer of the original model classifies 1000 classes, but we only have two, so we'll dispose of it, and add another final prediction layer (and one intermediate dense layer to achieve better accuracy).   

In [4]:
import org.jetbrains.kotlinx.dl.api.core.activation.Activations
import org.jetbrains.kotlinx.dl.api.core.initializer.HeNormal
import org.jetbrains.kotlinx.dl.api.core.layer.core.Dense


val frozenLayers = model.layers.dropLast(1)
    .onEach { it.isTrainable = false }

val layers = frozenLayers +
        Dense(
            name = "new_dense_1",
            kernelInitializer = HeNormal(),
            biasInitializer = HeNormal(),
            outputSize = 64,
            activation = Activations.Relu
        ) +
        Dense(
            name = "new_dense_2",
            kernelInitializer = HeNormal(),
            biasInitializer = HeNormal(),
            outputSize = 2,
            activation = Activations.Linear
        )

val transferedModel = Sequential.of(layers)

Finally, we can train this model. The only difference to training a model from scratch will be loading the weights for the frozen layers. 
These will not be further trained – that's how we can leverage the fact that this model has already learned some patterns on a much larger dataset.  

In [5]:
import org.jetbrains.kotlinx.dl.api.core.loss.Losses
import org.jetbrains.kotlinx.dl.api.core.metric.Metrics
import org.jetbrains.kotlinx.dl.api.core.optimizer.Adam
import org.jetbrains.kotlinx.dl.api.inference.keras.loadWeightsForFrozenLayers


transferedModel.use {

    it.compile(
        optimizer = Adam(),
        loss = Losses.SOFT_MAX_CROSS_ENTROPY_WITH_LOGITS,
        metric = Metrics.ACCURACY
    )

    val hdfFile = modelZoo.loadWeights()
    it.loadWeightsForFrozenLayers(hdfFile)

    val accuracyBeforeTraining = it.evaluate(dataset = test, batchSize = TEST_BATCH_SIZE).metrics[Metrics.ACCURACY]
    println("Accuracy before training $accuracyBeforeTraining")

    it.fit(
        dataset = train,
        batchSize = TRAINING_BATCH_SIZE,
        epochs = EPOCHS
    )

    val accuracyAfterTraining = it.evaluate(dataset = test, batchSize = TEST_BATCH_SIZE).metrics[Metrics.ACCURACY]

    println("Accuracy after training $accuracyAfterTraining")
}

No weights loading for input_1
No weights loading for block1_pool
No weights loading for block2_pool
No weights loading for block3_pool
No weights loading for block4_pool
No weights loading for block5_pool
No weights loading for flatten
Accuracy before training 0.3571428656578064
Accuracy after training 0.9285714626312256


---
**NOTE**

If you have obtained the pre-trained model's weights in HDF5 format differently from how it is described in this tutorial, 
you may not be able to load them with the default `loadWeightsForFrozenLayers` method.  


If the default method fails to load the weights from a *.h5 file, you can use the helper method 
`recursivePrintGroupInHDF5File(hdfFile, hdfFile)`  to inspect the HDF5 file structure and figure out the custom templates that are used to store the kernel and weights data. 
With those, you can use the `loadWeightsForFrozenLayersByPathTemplates` 
method to load nearly any Keras model weights stored in HDF5. 

Here's an example:
 
```kotlin
val kernelDataPathTemplate = "/%s/%s_W_1:0"
val biasDataPathTemplate = "/%s/%s_b_1:0"

it.loadWeightsForFrozenLayersByPathTemplates(hdfFile, kernelDataPathTemplate, biasDataPathTemplate)

```
--- 
That is it! Congratulations! You have learned how to use the transfer learning technique with KotlinDL.